In [ ]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, timezone
import snowflake.connector
import os
from dotenv import load_dotenv
import zipfile
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import locale

In [143]:
# Villes françaises avec coordonnées
cities = ['Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 
 'Strasbourg', 'Bordeaux', 'Lille', 'Rennes', 'Reims', 'Saint-Étienne', 
 'Le Havre', 'Toulon', 'Grenoble', 'Dijon', 'Angers', 'Nîmes', 'Villeurbanne']

In [90]:
#récupérer la lsite des documents a téléchager

# Lancer le navigateur (Chrome ici)
driver = webdriver.Chrome()

# Ouvre l'URL de ta page
driver.get("https://www.data.gouv.fr/datasets/resultats-du-controle-sanitaire-de-leau-distribuee-commune-par-commune/")

# Laisser un peu de temps pour charger la page
time.sleep(3)

# Récupérer tous les éléments avec la classe "space-y-2.5"
elements = driver.find_elements(By.CSS_SELECTOR, "div.border.border-gray-default.overflow-auto")

list_dis =[]
max_date_maj = datetime.strptime("2020-09-01 00:00:00", "%Y-%m-%d %H:%M:%S")

# Afficher leur texte
for  el in elements:
    
    dis = el.text
    if "dept" not in dis and ".pdf" not in dis :
        #extrait les dates de mis a jour des balise et les convertis en datetime
        date_maj = dis.split("\n")[1].split(" ")[4:7]
        date_maj ="/".join(date_maj)
        if "û" in date_maj :
            date_maj = date_maj.replace("û","Ã»")
        date_maj = datetime.strptime(date_maj, "%d/%B/%Y")

        #vérifie si la date du document à scrapper et plus récente que celle en bdd
        if date_maj > max_date_maj :
            print(dis.split("\n")[0], " " ,date_maj)

            try:
                # trouver le bouton <a> à l'intérieur de la div
                button = el.find_element(By.CSS_SELECTOR, "a.matomo_download")
                
                # Option 1 : cliquer directement
                button.click()
                time.sleep(2)  # petit délai pour le téléchargement
                
                # Option 2 : récupérer le lien et l'ajouter à la liste
                #href = button.get_attribute("href")
                #list_dis.append(href)
                #print(f"Document : {href}")
                
            except Exception as e:
                print(f"Élément : pas de bouton ou erreur - {e}")

time.sleep(300)

driver.quit()

dis-2025.zip   2025-09-01 00:00:00
dis-2024.zip   2025-03-28 00:00:00
dis-2023.zip   2024-10-14 00:00:00
dis-2022.zip   2023-07-07 00:00:00
dis-2021.zip   2023-08-11 00:00:00


In [94]:
# chemin vers ton fichier ZIP
zip_path = r"C:\Users\Utilisateur\Downloads"

list_donwload = os.listdir(zip_path)

for doc in list_donwload :
    if "dis-" in doc : 
        print (doc)


dis-2021.zip
dis-2022.zip
dis-2023.zip
dis-2024.zip
dis-2025.zip


In [118]:
# chemin vers le dossier download
download_path = r"C:\Users\Utilisateur\Downloads"

list_donwload = os.listdir(download_path)
list_df = []

for doc in list_donwload :
    if "dis-" in doc : 
        
        zip_path = f"{download_path}\{doc}"
        
        # ouvrir le ZIP
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # liste des fichiers contenus dans le ZIP
            for file_name in zip_ref.namelist():
                    if "PLV" in file_name :
                        with zip_ref.open(file_name) as f:
                            # adapte le séparateur selon ton fichier (',' ou '\t' ou ';')
                            df = pd.read_csv(f, sep=",")
                            list_df.append(df)

df_concat = pd.concat(list_df)

# Dictionnaire de renommage
rename_dict = {
    'dateprel' : 'date_plv',
    'nomcommuneprinc': 'city_name',
    'plvconformitereferencebact': 'conformite_ref_bact',
    'plvconformitereferencechim' : 'conformite_ref_chim',
    'plvconformitebacterio' : 'conformite_bact',
    'plvconformitechimique' : 'conformite_chim',
    'conclusionprel' :'conclusion_plv',
}

# Renommer les colonnes
df_concat = df_concat.rename(columns=rename_dict)

#reduire le dict au colonne désiré /renomé
df = df_concat[list(rename_dict.values())]

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20116\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep=",")
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20116\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep=",")
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20116\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep=",")
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20116\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f, sep=",")
C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20116\516815102.py:19: DtypeWarning: Columns (0,1,2,4,7) have mixed types. Specify dtype option on imp

In [ ]:
#clean city_name
def clean_cities(city) :
    if city == 'NIMES' :
         return 'Nîmes'
    if city == 'SAINT-ETIENNE' :
         return 'Saint-Étienne'
    if city == 'HAVRE (LE)' :
         return 'Le Havre'
    else :
         return city

def corel_city(row_city)   :
     for city in cities :
          if row_city.lower() in city.lower() :
               return city
          

df["city_name"] = df["city_name"].apply(clean_cities)
df["city_name"] = df["city_name"].apply(corel_city)

# Filtrer le DataFrame
df_reduce = df[df["city_name"].isin(cities)]

C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20116\3294417682.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["city_name"] = df["city_name"].apply(clean_cities)


['Paris' 'Nice' 'Marseille' 'Villeurbanne' 'Nantes' 'Dijon' 'Nîmes'
 'Bordeaux' 'Toulouse' 'Strasbourg' 'Montpellier' 'Rennes' 'Grenoble'
 'Angers' 'Saint-Étienne' 'Reims' 'Toulon' 'Lille' 'Lyon' 'Le Havre']


C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20116\3294417682.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["city_name"] = df["city_name"].apply(corel_city)


In [175]:
df_reduce

,date_plv,city_name,conformite_ref_bact,conformite_ref_chim,conformite_bact,conformite_chim,conclusion_plv
7623,2021-07-22,Paris,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
16119,2021-01-04,Nice,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
16124,2021-01-18,Nice,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
16129,2021-02-01,Nice,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
16132,2021-02-15,Nice,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
...,...,...,...,...,...,...,...
211099,2025-07-29,Toulon,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
231194,2025-03-25,Toulouse,N,C,C,C,Eau conforme aux limites de qualité réglementa...
231195,2025-04-01,Toulouse,C,C,C,C,Eau d'alimentation conforme aux exigences de q...
231196,2025-03-25,Toulouse,S,C,S,C,Eau d'alimentation conforme aux exigences de q...
